In [ ]:
import torch
import torch.nn as nn

# Define the convolutional block
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, batch_norm=True):
        super(ConvBlock, self).__init__()
        layers = []
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
        if batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.1, inplace=True))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

# Define the residual block used in Darknet-53
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvBlock(in_channels, out_channels // 2, stride=1)
        self.conv2 = ConvBlock(out_channels // 2, out_channels, stride=1, batch_norm=True)

    def forward(self, x):
        return x + self.conv2(self.conv1(x))

# Define the Darknet-53 backbone
class Darknet53(nn.Module):
    def __init__(self):
        super(Darknet53, self).__init__()
        self.layer1 = self._make_layer(3, 32, num_blocks=1)
        self.layer2 = self._make_layer(32, 64, num_blocks=2)
        self.layer3 = self._make_layer(64, 128, num_blocks=8)
        self.layer4 = self._make_layer(128, 256, num_blocks=8)
        self.layer5 = self._make_layer(256, 512, num_blocks=4)
        self.layer6 = self._make_layer(512, 1024, num_blocks=4)

        self.conv_final = ConvBlock(1024, 1024, stride=2)

    def _make_layer(self, in_channels, out_channels, num_blocks):
        layers = [ConvBlock(in_channels, out_channels, stride=2)]
        for _ in range(num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.conv_final(x)
        return x
